In [ ]:
#| hide
#| default_exp utils

# Utils

## Disk Cache

In [ ]:
#| export
# dill is an improved version of pickle, using it to support namedtuples
import dill
from pathlib import Path
import inspect
import hashlib

In [ ]:
#| hide
def a():
    # comment
    return 'a'
inspect.getsource(a)

"def a():\n    # comment\n    return 'a'\n"

In [ ]:
#| export
# inspired from https://gist.github.com/shantanuo/c6a376309d6bac6bd55bf77e3961b5fb
def cache_disk(base_file, rm_cache=False):
    "Decorator to cache function output to disk"
    base_file = Path(base_file)
    def decorator(original_func):
        
        f_hash = hashlib.md5(inspect.getsource(original_func).encode()).hexdigest()
        filename = base_file.parent / (base_file.stem + f_hash + ".pickle")
        
        if rm_cache: filename.unlink()
        
        try:
            cache = dill.load(open(filename, 'rb'))
        except (IOError, ValueError):
            cache = {}

        def save_data():
            dill.dump(cache, open(filename, "wb"))  

        def new_func(*args):
            if tuple(args) not in cache:
                cache[tuple(args)] = original_func(*args)
                save_data()
            return cache[args]

        return new_func

    return decorator

In [ ]:
import time
from tempfile import tempdir

In [ ]:
cp = Path(tempdir) / "test_cache"

In [ ]:
@cache_disk(cp)
def slow_add(a,b):
    time.sleep(1)
    return a + b 

this time is the first time so not from the cache

In [ ]:
%time slow_add(1,2)

CPU times: user 3.58 ms, sys: 88 µs, total: 3.67 ms
Wall time: 1 s


3

now is much faster beacuse of the cache

In [ ]:
%time slow_add(1,2)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


3

adding comments change the hash, so the function is still cached

In [ ]:
@cache_disk(cp)
def slow_add(a,b):
    time.sleep(1)
    # this is a comment
    return a + b 

In [ ]:
%time slow_add(1,2)

CPU times: user 45 µs, sys: 976 µs, total: 1.02 ms
Wall time: 1 s


3

In [ ]:
%time slow_add(1,2)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


3

In [ ]:
#| export
import torch
import numpy as np

In [ ]:
#| export
def reset_seed(seed=27):
    torch.manual_seed(seed)
    np.random.seed(seed)

In [ ]:
#| export
def reset_seed(seed=27):
    torch.manual_seed(seed)
    np.random.seed(seed)

## Render Docstring

renders a ReStructured text docstring in a notebook cell



In [ ]:
#| export
# from IPython.core.magic import register_cell_magic, register_line_magic
from IPython.display import HTML
import docutils.core

In [ ]:
def rstdoc(obj):
    "Render `__doc__` as ReStructuredText"
    html = docutils.core.publish_parts(obj.__doc__, writer_name='html5')['fragment']
    return HTML(html)

In [ ]:
rstdoc(docutils.core.publish_parts)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()